In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
if os.getenv("GOOGLE_API_KEY"):
    print("Google API key loaded successfully")
else:
    print("Google API Key not found")

Google API key loaded successfully


In [12]:

##  File Management Agent Implementation

# Cell 1: Import dependencies and setup
from dotenv import load_dotenv
import os
import json
import google.generativeai as genai
from datetime import datetime

# Load environment variables
load_dotenv()

# Configure Gemini API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

print("Environment setup complete!")

# Cell 2: Helper Functions
def list_files():
    """List all files in the current directory."""
    files = [f for f in os.listdir(".") if os.path.isfile(f)]
    return files

def read_file(file_name: str):
    """Read the content of a file. Return error if not found."""
    try:
        with open(file_name, "r", encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return f"File '{file_name}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"

def write_file(file_name: str, content: str):
    """Write content to a file."""
    try:
        with open(file_name, "w", encoding='utf-8') as f:
            f.write(content)
        return f"Successfully wrote to '{file_name}'"
    except Exception as e:
        return f"Error writing file: {str(e)}"

def delete_file(file_name: str):
    """Delete a file."""
    try:
        os.remove(file_name)
        return f"Successfully deleted '{file_name}'"
    except FileNotFoundError:
        return f"File '{file_name}' not found."
    except Exception as e:
        return f"Error deleting file: {str(e)}"

def update_file(file_name: str, content: str):
    """Update/append content to a file."""
    try:
        with open(file_name, "a", encoding='utf-8') as f:
            f.write(content)
        return f"Successfully updated '{file_name}'"
    except Exception as e:
        return f"Error updating file: {str(e)}"

def create_docs_directory():
    """Create docs directory if it doesn't exist."""
    if not os.path.exists("docs"):
        os.makedirs("docs")
        return "Created 'docs' directory"
    return "'docs' directory already exists"

print("Helper functions defined!")

# Cell 3: AI Response Processing
def extract_markdown_block(text: str, block_type: str) -> str:
    """Extract content from markdown code block."""
    start = text.find("```" + block_type)
    end = text.find("```", start + 1)
    if start != -1 and end != -1:
        return text[start + len(block_type) + 3:end].strip()
    return text.strip()

def generate_response(prompt: str) -> str:
    """Send prompt to Gemini and return response."""
    try:
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f" Error generating response: {str(e)}"

def parse_action(response: str) -> dict:
    """Parse LLM response into action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "Invalid format. Missing tool_name or args."}}
    except Exception as e:
        return {"tool_name": "error", "args": {"message": f"Parse error: {str(e)}"}}

print("AI processing functions defined!")

# Cell 4: Agent Configuration
# Agent memory and settings
memory = []
iterations = 0
max_iterations = 10

# System rules for the file management agent
agent_rules = [{
    "role": "system",
    "content": """
You are a File Management AI Agent that can perform various file operations and create documentation.

Available tools:
- list_files() -> List[str]: List all files in the current directory
- read_file(file_name: str) -> str: Read content of a specific file  
- write_file(file_name: str, content: str): Write content to a file
- delete_file(file_name: str): Delete a specific file
- update_file(file_name: str, content: str): Append content to a file
- create_docs_directory(): Create a 'docs' directory
- generate_documentation(file_names: List[str]): Generate documentation for specified files
- terminate(message: str): End the agent and provide summary

Special Capabilities:
- When asked to create documentation, read the specified files and create comprehensive docs
- Save documentation in the 'docs' directory
- Can analyze code, text files, and generate meaningful documentation
- Can manage file operations like create, read, update, delete

Every response MUST contain an action in this format:

```action
{
    "tool_name": "tool_name_here",
    "args": {
        "arg1": "value1",
        "arg2": "value2"
    }
}

Be helpful, thorough, and always confirm actions taken.
"""
}]

print(" Agent configuration complete!")

# Cell 5: Advanced Tool Functions
def generate_documentation(file_names: list):
    """Generate comprehensive documentation for specified files."""
    try:
        create_docs_directory()  # Ensure docs directory exists
        
        documentation = f"# Documentation Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
        
        for file_name in file_names:
            if os.path.exists(file_name):
                content = read_file(file_name)
                if not content.startswith("X"):
                    # Generate documentation using AI
                    doc_prompt = f"""
                    Analyze this file and create comprehensive documentation:
                    
                    File: {file_name}
                    Content: {content}
                    
                    Create documentation that includes:
                    1. Purpose and overview
                    2. Key components/functions
                    3. Usage examples if applicable
                    4. Dependencies if any
                    5. Notes and important details
                    
                    Format as markdown.
                    """
                    
                    ai_doc = generate_response(doc_prompt)
                    documentation += f"## {file_name}\n\n{ai_doc}\n\n---\n\n"
                else:
                    documentation += f"## {file_name}\n\n Could not read file\n\n---\n\n"
            else:
                documentation += f"## {file_name}\n\n File not found\n\n---\n\n"
        
        doc_file_name = f"docs/documentation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
        write_result = write_file(doc_file_name, documentation)
        
        return f" Documentation generated and saved to '{doc_file_name}'. {write_result}"
        
    except Exception as e:
        return f" Error generating documentation: {str(e)}"

print(" Advanced functions defined!")

# Cell 6: Main Agent Loop
def run_agent():
    global memory, iterations
    
    print(" File Management AI Agent Started!")
    print("Commands you can try:")
    print("- 'list all files'")
    print("- 'read filename.txt'") 
    print("- 'create documentation for file1.py, file2.txt'")
    print("- 'delete filename.txt'")
    print("- 'quit' or 'exit' to stop")
    print("-" * 50)
    
    while iterations < max_iterations:
        try:
            # Get user input
            user_input = input(f"\n[{iterations + 1}] You: ")
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'stop']:
                print("Agent stopped by user.")
                break
                
            memory.append({"role": "user", "content": user_input})
            
            # Build prompt
            prompt = agent_rules[0]["content"] + "\n\nConversation History:\n" + json.dumps(memory[-10:])  # Last 10 messages
            
            print(" Agent thinking...")
            response = generate_response(prompt)
            print(f"Agent: {response}")
            
            # Parse and execute action
            action = parse_action(response)
            result = execute_action(action)
            
            print(f"⚡ Result: {result}")
            
            # Update memory
            memory.extend([
                {"role": "assistant", "content": response},
                {"role": "user", "content": json.dumps(result)}
            ])
            
            # Check for termination
            if action["tool_name"] == "terminate":
                break
                
            iterations += 1
            
        except KeyboardInterrupt:
            print("\n Agent interrupted by user.")
            break
        except Exception as e:
            print(f"❌ Error in agent loop: {str(e)}")
            break
    
    print(f"\nAgent completed {iterations} iterations.")
    return memory

def execute_action(action: dict):
    """Execute the parsed action."""
    tool_name = action["tool_name"]
    args = action.get("args", {})
    
    try:
        if tool_name == "list_files":
            return {"result": list_files()}
        elif tool_name == "read_file":
            return {"result": read_file(args["file_name"])}
        elif tool_name == "write_file":
            return {"result": write_file(args["file_name"], args["content"])}
        elif tool_name == "delete_file":
            return {"result": delete_file(args["file_name"])}
        elif tool_name == "update_file":
            return {"result": update_file(args["file_name"], args["content"])}
        elif tool_name == "create_docs_directory":
            return {"result": create_docs_directory()}
        elif tool_name == "generate_documentation":
            return {"result": generate_documentation(args["file_names"])}
        elif tool_name == "terminate":
            return {"result": args["message"]}
        elif tool_name == "error":
            return {"error": args["message"]}
        else:
            return {"error": f"Unknown tool: {tool_name}"}
    except Exception as e:
        return {"error": f"Execution error: {str(e)}"}

print("Agent loop functions defined!")

# Cell 7: Start the Agent
# Run the agent
conversation_history = run_agent()

# Cell 8: Test Functions (Optional)
# Test individual functions
print("Testing functions...")
print("Files:", list_files())
print("Docs directory:", create_docs_directory())

# Create a sample file to test with
sample_code = '''def hello_world():
    """A simple hello world function."""
    print("Hello, World!")
    return "Hello, World!"

if __name__ == "__main__":
    hello_world()
'''

write_file("sample.py", sample_code)
print("Sample file created for testing")

## Usage Instructions:


   


Environment setup complete!
Helper functions defined!
AI processing functions defined!
 Agent configuration complete!
 Advanced functions defined!
Agent loop functions defined!
 File Management AI Agent Started!
Commands you can try:
- 'list all files'
- 'read filename.txt'
- 'create documentation for file1.py, file2.txt'
- 'delete filename.txt'
- 'quit' or 'exit' to stop
--------------------------------------------------



[1] You:  Delete Untitled.ipynb


 Agent thinking...
Agent: ```action
{
  "tool_name": "delete_file",
  "args": {
    "file_name": "Untitled.ipynb"
  }
}
```

File action confirmed: File 'Untitled.ipynb' deleted.

⚡ Result: {'result': "Successfully deleted 'Untitled.ipynb'"}



[2] You:  list all files


 Agent thinking...
Agent: ```action
{
  "tool_name": "list_files",
  "args": {}
}
```
```tool_code
File listing operation initiated.
```

⚡ Result: {'result': ['file-management-ai-agent.ipynb', '.env', 'sample.py']}



[3] You:  quit


Agent stopped by user.

Agent completed 2 iterations.
🧪 Testing functions...
Files: ['file-management-ai-agent.ipynb', '.env', 'sample.py']
Docs directory: 'docs' directory already exists
Sample file created for testing
